In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt

from Miniproject_2.model import Model, Relu, Sigmoid, Sequential, MSELoss

import torch
from torch.nn import functional as F


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

valid_input, valid_target = torch.load('../val_data.pkl',map_location=device)#validation set (noise-clean)
train_input, train_target = torch.load('../train_data.pkl',map_location=device) #test set (noise-noise)

num_samples = 1000
valid_input=torch.narrow(valid_input,0,0,num_samples)
valid_target=torch.narrow(valid_target,0,0,num_samples)
train_input=torch.narrow(train_input,0,0,num_samples)
train_target=torch.narrow(train_target,0,0,num_samples)

print("Vector shape: ",train_input.shape)

Vector shape:  torch.Size([1000, 3, 32, 32])


In [3]:
idx = 22
len = 10
input=valid_input[idx].float()
target=valid_target[idx].float()

# fig,ax = plt.subplots(1,2, figsize=(8,8))
# ax[0].imshow(input.permute(1,2,0))
# ax[1].imshow(target.permute(1,2,0))
out = MSELoss().forward(input,target)

print(out)

out_true = F.mse_loss(input,target, reduction='mean')
print(out_true)

torch.Size([10, 3, 32, 32])
tensor(611.3060)
tensor(611.3060)


In [4]:
mod_dict={
    'relu1' : Relu(),
    'Sig1' : Sigmoid()
}

seq1 = Sequential(**mod_dict)

inp=torch.rand(10)-0.5
seq1(inp)


input:  tensor([ 0.0760,  0.0821, -0.2584,  0.0739,  0.0804, -0.4182,  0.1903, -0.4390,
        -0.2009, -0.2833])
relu1 tensor([0.0760, 0.0821, 0.0000, 0.0739, 0.0804, 0.0000, 0.1903, 0.0000, 0.0000,
        0.0000])
Sig1 tensor([0.5190, 0.5205, 0.5000, 0.5185, 0.5201, 0.5000, 0.5474, 0.5000, 0.5000,
        0.5000])


tensor([0.5190, 0.5205, 0.5000, 0.5185, 0.5201, 0.5000, 0.5474, 0.5000, 0.5000,
        0.5000])

In [2]:
inp=torch.rand(10)-0.5
print('inp: ',inp)


out1=Relu().forward(inp)
print('out1: ', out1)
out2=Sigmoid().forward(inp)
print('out2: ', out2)

inp:  tensor([-0.4169, -0.1908, -0.2506, -0.3950, -0.4416, -0.4296, -0.3108,  0.2979,
         0.1155,  0.2701])
out1:  tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2979, 0.1155,
        0.2701])
out2:  tensor([0.3973, 0.4524, 0.4377, 0.4025, 0.3914, 0.3942, 0.4229, 0.5739, 0.5289,
        0.5671])
